# Exercise 3
### A convex optimization problem.

In [2]:
def f0(x):
    return x[0]*x[0] + x[1]*x[1]

f1 = lambda x: -x[0]*x[0] - x[0]*x[1] - x[1]*x[1] + 3     
f2 = lambda x: 3*x[0] + 2*x[1] - 3     

fs = [f1, f2]

cons = (
    {'type': 'ineq', 'fun': f1},
    {'type': 'ineq', 'fun': f2},
)

## Solve it using scypy
Don't forget to check the convergence (whatever that means)

In [8]:
import scipy.optimize as opt
init = [[0,0], [1,1], [30, -30], [-300, 300], [-30000, 30000], [999999, 1]]
sols = [opt.minimize(f0, x_0, method='SLSQP', constraints=cons) for x_0 in init]
print([s.fun for s in sols])
print([s.nit for s in sols])
print([s.nfev for s in sols])
print(sols[0].x)

[0.6923076923076928, 0.6923076923076908, 0.6923081546863251, 0.6923076923643912, 0.6923076923643744, 0.6923076931710144]
[3, 4, 4, 4, 4, 6]
[10, 12, 13, 15, 15, 18]
[0.69230769 0.46153846]


## Solution with CVX toolbox

In [11]:
import cvxpy as cp 

x = cp.Variable(2)
Q = [[1, .5], [.5, 1]]
constraints = [cp.quad_form(x, Q) <= 3,
               3*x[0] + 2*x[1] >= 3]
prob = cp.Problem(cp.Minimize(x[0]**2 + x[1]**2),
                  constraints=constraints)
prob.solve()

0.6923076926895174

## Get info from the dual problem

In [6]:
print("optimal constraint 1 dual variable", constraints[0].dual_value)
print("optimal constraint 2 dual variable", constraints[1].dual_value)
print("[x1, x2] value:", x.value)

optimal constraint 1 dual variable [7.23394671e-10]
optimal constraint 2 dual variable 0.461550969108317
[x1, x2] value: [0.69231361 0.46152958]


## The error that I got

In [12]:
import cvxpy as cp 

x = cp.Variable(2)
constraints = [x[0]**2 + x[0]*x[1] + x[1]**2 <= 3,
               3*x[0] + 2*x[1] >= 3]
prob = cp.Problem(cp.Minimize(x[0]**2 + x[1]**2), constraints=constraints)
prob.solve()

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
power(var321[0], 2.0) + var321[0] @ var321[1] + power(var321[1], 2.0) <= 3.0 , because the following subexpressions are not:
|--  var321[0] @ var321[1]

In [17]:
from sympy import *

x1, x2 = symbols('x1 y2')
x = [x1, x2]
f0 = x[0]*x[0] + x[1]*x[1]
h1 = +x[0]*x[0] + x[0]*x[1] + x[1]*x[1] - 3     
h2 = -3*x[0] - 2*x[1] + 3     
h = [h1, h2]
init_printing(use_latex=true)
J = Matrix([simplify(diff(f0, xi)) for xi in x])
l1, l2 = symbols('λ_1, λ_2')
L = f0 + l1*h1 + l2 * h2
JL = Matrix([simplify(diff(L, xi)) for xi in x])
HL = Matrix([simplify(diff(JL.T, xi)) for xi in x])
print(latex(JL))

\left[\begin{matrix}2 x_{1} + λ_{1} \cdot \left(2 x_{1} + y_{2}\right) - 3 λ_{2}\\2 y_{2} + λ_{1} \left(x_{1} + 2 y_{2}\right) - 2 λ_{2}\end{matrix}\right]
